<a href="https://colab.research.google.com/github/ethankyle360/ML4SupplyChain/blob/main/section_10_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment: Manipulate data to get weekly sales of a UK retailer, you will need the month and trend to fit a linear regression model.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
retail_clean = pd.read_csv('retail_clean.csv')

In [4]:
retail_clean.info()
retail_clean.InvoiceDate
retail_clean['InvoiceDate'] = pd.to_datetime(retail_clean['InvoiceDate'])
retail_clean['date'] = retail_clean['InvoiceDate'].dt.strftime('%Y-%m-%d')
retail_clean['date'] = pd.to_datetime(retail_clean['date'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679373 entries, 0 to 679372
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   679373 non-null  int64  
 1   Invoice      679373 non-null  int64  
 2   StockCode    679373 non-null  object 
 3   Description  679373 non-null  object 
 4   Quantity     679372 non-null  float64
 5   InvoiceDate  679372 non-null  object 
 6   Price        679372 non-null  float64
 7   Customer ID  679372 non-null  float64
 8   Country      679372 non-null  object 
 9   Revenue      679372 non-null  float64
dtypes: float64(4), int64(2), object(4)
memory usage: 51.8+ MB


In [6]:
retail_clean['month'] = retail_clean.date.dt.month
retail_clean['year'] = retail_clean.date.dt.year
retail_clean['week'] = retail_clean.date.dt.isocalendar().week # Use .dt.isocalendar().week to extract week number

In [8]:
retail_clean.columns
retail_clean.month.describe()
time_series = retail_clean.groupby('date').agg(total_sales = ('Quantity', np.sum)).reset_index().sort_values('date')